In [30]:
import numpy as np
import random 
from grid_mdp import Grid_Mdp

grid = Grid_Mdp()
states = grid.states
actions = grid.actions
gamma = grid.gamma

### epsilon-greedy

$  \varepsilon-greedy $

$ \varPi_{\varepsilon-greedy}(s,a) = $

In [63]:
def epsilon_greedy(qfunc, state, epsilon):
    
    amax = 0
    key = '{}_{}'.format(state, actions[0])
    qmax = qfunc[key]
    for i in range(len(actions)):
        key = '{}_{}'.format(state, actions[i])
        q = qfunc[key]
        if qmax < q:
            qmax = q
            amax = i
    
    pro = [ epsilon/len(actions) for j in range(len(actions))]
    pro[amax] += (1 - epsilon)
    
    
    return np.random.choice(actions, p = pro)

### Monte Carlo

$ q(s,a) = \frac{q(s,a) \times n(s,a)+g}{n(s,a)+1} $    
$ n(s,a) = n(s,a)+1 $

In [83]:
def mc(num_iters, epsilon):
    
    qfunc, n = dict(), dict()
    
    for s in states:
        for a in actions:
            qfunc['{}_{}'.format(s,a)] = 0.0
            n['{}_{}'.format(s,a)] = 1e-3
            
    for iter1 in range(num_iters):
      
        s_sample = []
        a_sample = []
        r_sample = []
        
        s = states[ random.randint(0,len(states)-1) ]
        t = False

        while False ==t:
            
            a = epsilon_greedy(qfunc, s, epsilon)
            t, s1, r = grid.transform(s,a)
            
            s_sample.append(s)
            r_sample.append(r)
            a_sample.append(a)
            
            s = s1

        
        
        g = 0.0
        for i in range(len(s_sample)-1, -1, -1):
            g *= gamma
            g += r_sample[i]
            
        for i in range(len(s_sample)):
            
            key = '{}_{}'.format(s_sample[i], a_sample[i])
            n[key] += 1.0
            qfunc[key] = (qfunc[key] * n[key] +g) / (n[key]+1)
            
            g -= r_sample[i]
            g /= gamma
            
    return qfunc

In [108]:
mc(100, 0.7)

{'1_e': 0.2430446954785559,
 '1_n': -0.21119421555688866,
 '1_s': -0.7998400319936013,
 '1_w': -0.13856552276423573,
 '2_e': 0.47699181186422884,
 '2_n': 0.27889032886126874,
 '2_s': 0.24220545510299324,
 '2_w': 0.0889152605446803,
 '3_e': 0.24604555054032126,
 '3_n': 0.4686035398735664,
 '3_s': 0.9777560498655586,
 '3_w': 0.27354124718293577,
 '4_e': -0.04371578034648647,
 '4_n': 0.12191063179922283,
 '4_s': 0.19311625624958345,
 '4_w': 0.48524163582139834,
 '5_e': -0.1251262988277974,
 '5_n': -0.1696717495299066,
 '5_s': -0.8887901344295079,
 '5_w': 0.18405047759363066,
 '6_e': 0.0,
 '6_n': 0.0,
 '6_s': 0.0,
 '6_w': 0.0,
 '7_e': 0.0,
 '7_n': 0.0,
 '7_s': 0.0,
 '7_w': 0.0,
 '8_e': 0.0,
 '8_n': 0.0,
 '8_s': 0.0,
 '8_w': 0.0}

### SARSA

$ q(s,a) = q(s,a) + \alpha(r + \gamma q(s^{'}, a^{'}) - q(s,a))$

In [92]:
def sarsa(num_iters, alpha, epsilon):
    
    qfunc = dict()
    for s in states:
        for a in actions:
            qfunc['{}_{}'.format(s,a)]  = 0.0
            
    for _ in range(num_iters):
        
        s = states[ random.randint(0, len(states)-1) ]
        a = actions[ random.randint(0, len(actions)-1)]
        t = False
        
        while t == False:
            
            key = '{}_{}'.format(s, a)
            t,s1, r = grid.transform(s,a)
            
            a1 = epsilon_greedy(qfunc, s1, epsilon)
            key1 = '{}_{}'.format(s1, a1)
            
            qfunc[key] = qfunc[key] + alpha * ( r + gamma* qfunc[key1] - qfunc[key])
            
            s = s1
            a = a1
    
    return qfunc

In [110]:
sarsa(100, 0.2, 0.7)

{'1_e': 0.1277344058361311,
 '1_n': -0.12026583181688834,
 '1_s': -0.83222784,
 '1_w': -0.07253954582387098,
 '2_e': 0.5789356808936588,
 '2_n': 0.16756323053346622,
 '2_s': 0.18432342626581066,
 '2_w': -0.0811167645696,
 '3_e': 0.09506872173096774,
 '3_n': 0.44761984028649116,
 '3_s': 0.9998338465005269,
 '3_w': 0.018961708224523477,
 '4_e': 0.032245768488882776,
 '4_n': 0.04506146498795313,
 '4_s': 0.2607755258647055,
 '4_w': 0.7146935272470093,
 '5_e': 0.08656918069164049,
 '5_n': -0.0723208168205659,
 '5_s': -0.8926258176,
 '5_w': 0.29509967573980517,
 '6_e': 0.0,
 '6_n': 0.0,
 '6_s': 0.0,
 '6_w': 0.0,
 '7_e': 0.0,
 '7_n': 0.0,
 '7_s': 0.0,
 '7_w': 0.0,
 '8_e': 0.0,
 '8_n': 0.0,
 '8_s': 0.0,
 '8_w': 0.0}

### Q-Learning

$ q(s,a) = q(s,a) + \alpha(r + \gamma max_{a^{'}}q(s^{'}, a^{'}) - q(s,a))$

In [100]:
def qlearning(num_iters, alpha, epsilon):
        
    qfunc = dict()
    for s in states:
        for a in actions:
            qfunc['{}_{}'.format(s,a)]  = 0.0
    
    for _ in range(num_iters):
        
        s = states[ random.randint(0, len(states)-1)]
        a = actions[ random.randint(0, len(actions)-1)]
        t = False
        
        while False == t:
            
            key = '{}_{}'.format(s,a)
            t,s1,r = grid.transform(s,a)
            
            key1 = ''
            qmax = -1.0
            
            for a_tmp in actions:
                
                key_tmp = '{}_{}'.format(s1, a_tmp)
                if qmax < qfunc[ key_tmp]:
                    qmax =  qfunc[ key_tmp]
                    key1 = key_tmp
                
                
            qfunc[key] = qfunc[key] + alpha * (r+ gamma* qfunc[key1] - qfunc[key])
            
            s = s1
            a = epsilon_greedy(qfunc, s1, epsilon)
            
    return qfunc

In [111]:
qlearning(100, 0.2, 0.7)

{'1_e': 0.5976707727283048,
 '1_n': 0.20905893932053274,
 '1_s': -0.9141006540800001,
 '1_w': 0.3570747585630884,
 '2_e': 0.7971046677749101,
 '2_n': 0.49745329138468186,
 '2_s': 0.5311845882989159,
 '2_w': 0.3880783487935838,
 '3_e': 0.6106034390900863,
 '3_n': 0.764062672379765,
 '3_s': 0.9997403851570732,
 '3_w': 0.5990235023868303,
 '4_e': 0.38318574583296433,
 '4_n': 0.5305703324591649,
 '4_s': 0.5557555166374503,
 '4_w': 0.796008537146206,
 '5_e': 0.29245867977921114,
 '5_n': 0.28837046061858734,
 '5_s': -0.9141006540800001,
 '5_w': 0.5776525743039942,
 '6_e': 0.0,
 '6_n': 0.0,
 '6_s': 0.0,
 '6_w': 0.0,
 '7_e': 0.0,
 '7_n': 0.0,
 '7_s': 0.0,
 '7_w': 0.0,
 '8_e': 0.0,
 '8_n': 0.0,
 '8_s': 0.0,
 '8_w': 0.0}